In [55]:
import pandas as pd
import random
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Codes available at https://github.com/udellgroup/online_mixed_gc_imp
sys.path.append("/Users/annalaufey/online_mixed_gc_imp")
from em.expectation_maximization import ExpectationMaximization

In [97]:
data = pd.read_csv("national.csv") 
#this will give a warning, which I deal with further down.

/Applications/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (27,28,34,35,45,74,85,217,220,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
data.head(n=5)

,sitecode,sitename,sitetype,sitetypenum,year,survyear,weight,stratum,PSU,record,...,qnwenthungry,qnmusclestrength,qnsunscreenuse,qnindoortanning,qnsunburn,qnconcentrating,qncurrentasthma,qnwheresleep,qnspeakenglish,qntransgender
0,XX,United States (XX) ...,National ...,3,1991,1,0.2645,12210,5,3537.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XX,United States (XX) ...,National ...,3,1991,1,0.5060,12310,29,3538.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XX,United States (XX) ...,National ...,3,1991,1,0.3201,11400,33,9423.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XX,United States (XX) ...,National ...,3,1991,1,0.7787,11200,21,11716.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XX,United States (XX) ...,National ...,3,1991,1,1.3896,12100,8,13693.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# replace all empty spaces with NaNs
data = data.replace(' ', np.NaN)

In [99]:
del data['sitecode']
del data['sitename']
del data['sitetype']
del data['sitetypenum']
del data['survyear']
del data['weight']
del data['stratum']
del data['PSU']
del data['record']

In [100]:
# Converting all data from strings
data = data.fillna(0).astype(int)
data = data.replace(0, np.NaN)

In [103]:
data.to_csv('data.csv')

In [101]:
# missing ratio at each column
misrat = data.apply(lambda x: np.mean(np.isnan(x)))
misrat

year                0.000000
age                 0.003271
sex                 0.004182
grade               0.006055
race4               0.015119
race7               0.023829
stheight            0.315299
stweight            0.315299
bmi                 0.308622
bmipct              0.317217
qnobese             0.310992
qnowt               0.310992
q66                 0.807739
q65                 0.831978
sexid               0.807739
sexid2              0.807739
sexpart             0.833238
sexpart2            0.833238
q8                  0.037559
q9                  0.012455
q10                 0.758061
q11                 0.755066
q12                 0.052986
q13                 0.072729
q14                 0.884724
q15                 0.071400
q16                 0.062460
q17                 0.052779
q18                 0.074271
q19                 0.360178
                      ...   
qnillict            0.436928
qngenderexp         1.000000
qntaughtHIV         0.222439
qntaughtsexed 

In [102]:
# Getting rid of columns where 100% of values are missing.
getrid = misrat[misrat == 1.0]
for i in getrid.index:
    del data[i]

print("The new dimensions are ", data.shape)
data.apply(lambda x: np.mean(np.isnan(x)))

The new dimensions are  (217340, 258)


year                 0.000000
age                  0.003271
sex                  0.004182
grade                0.006055
race4                0.015119
race7                0.023829
stheight             0.315299
stweight             0.315299
bmi                  0.308622
bmipct               0.317217
qnobese              0.310992
qnowt                0.310992
q66                  0.807739
q65                  0.831978
sexid                0.807739
sexid2               0.807739
sexpart              0.833238
sexpart2             0.833238
q8                   0.037559
q9                   0.012455
q10                  0.758061
q11                  0.755066
q12                  0.052986
q13                  0.072729
q14                  0.884724
q15                  0.071400
q16                  0.062460
q17                  0.052779
q18                  0.074271
q19                  0.360178
                       ...   
qcurrentasthma       0.800179
qspeakenglish        0.860532
qnbikehelm

In [49]:
# number of unique values for each column
data.apply(lambda x: len(x.unique()))

year                  15
age                    8
sex                    3
grade                  5
race4                  5
race7                  8
stheight               3
stweight             154
bmi                   70
bmipct               100
qnobese                3
qnowt                  3
q66                    5
q65                    5
sexid                  5
sexid2                 4
sexpart                5
sexpart2               4
q8                     6
q9                     6
q10                    7
q11                    9
q12                    6
q13                    6
q14                    6
q15                    6
q16                    9
q17                    9
q18                    9
q19                    3
                    ... 
qcurrentasthma         5
qspeakenglish          5
qnbikehelmet           3
qndrivemarijuana       3
qnpropertydamage       3
qncigschool            3
qnchewtobschool        3
qnalcoholschool        3
qnhowmarijuana         3


In [51]:
dat = data.iloc[:,0:98]
dat.apply(lambda x: np.mean(np.isnan(x)))

year        0.000000
age         0.003271
sex         0.004182
grade       0.006055
race4       0.015119
race7       0.023829
stheight    0.315299
stweight    0.315299
bmi         0.308622
bmipct      0.317217
qnobese     0.310992
qnowt       0.310992
q66         0.807739
q65         0.831978
sexid       0.807739
sexid2      0.807739
sexpart     0.833238
sexpart2    0.833238
q8          0.037559
q9          0.012455
q10         0.758061
q11         0.755066
q12         0.052986
q13         0.072729
q14         0.884724
q15         0.071400
q16         0.062460
q17         0.052779
q18         0.074271
q19         0.360178
              ...   
q58         0.078071
q59         0.075090
q60         0.077726
q61         0.077528
q62         0.080551
q63         0.084747
q64         0.091433
q67         0.023097
q68         0.046673
q69         0.275720
q70         0.274459
q71         0.284563
q72         0.286197
q73         0.286560
q74         0.287053
q75         0.545818
q76         0

In [62]:
del dat['sex']
del dat['race4']
del dat['race7']
del dat['sexid']
del dat['sexid2']
del dat['sexpart']
del dat['sexpart2']

In [82]:
ind = random.sample(range(0,217340), 2500)
dati = dat.iloc[ind,:]
dati.apply(lambda x: np.mean(np.isnan(x)))

year        0.0000
age         0.0024
grade       0.0052
stheight    0.3068
stweight    0.3068
bmi         0.3008
bmipct      0.3100
qnobese     0.3028
qnowt       0.3028
q66         0.8060
q65         0.8272
q8          0.0372
q9          0.0140
q10         0.7540
q11         0.7520
q12         0.0412
q13         0.0736
q14         0.8876
q15         0.0724
q16         0.0636
q17         0.0428
q18         0.0728
q19         0.3516
q20         0.8880
q21         0.7548
q22         0.7424
q23         0.5932
q24         0.6620
q25         0.2548
q26         0.0120
             ...  
q58         0.0736
q59         0.0728
q60         0.0744
q61         0.0756
q62         0.0792
q63         0.0812
q64         0.0936
q67         0.0200
q68         0.0448
q69         0.2660
q70         0.2640
q71         0.2764
q72         0.2780
q73         0.2788
q74         0.2784
q75         0.5452
q76         0.3104
q77         0.7064
q78         0.6612
q79         0.2684
q80         0.4148
q81         

In [83]:
ord_indices = dati.apply(lambda x: len(x.unique())) < 10
cont_indices = ~ord_indices

In [84]:
datainput = dati.values # transform to numpy array

In [85]:
gc = ExpectationMaximization()
Zimp, Corr_est = gc.impute_missing(datainput, cont_indices=cont_indices, ord_indices=ord_indices, verbose=True)

/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu


Copula correlation change ratio:  0.4051


/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu


Copula correlation change ratio:  0.1735


/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu


...

Copula correlation change ratio:  0.0324


/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4861: RuntimeWarning: divide by zero encountered in true_divide
  mu = (pA - pB) / d   # correction sign
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_dis

Copula correlation change ratio:  0.0359


/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4862: RuntimeWarning: invalid value encountered in multiply
  mu2 = 1 + (a*pA - b*pB) / d - mu*mu
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_distns.py:4861: RuntimeWarning: divide by zero encountered in true_divide
  mu = (pA - pB) / d   # correction sign
/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/_continuous_dis

Copula correlation change ratio:  0.0346
Convergence not achieved at maximum iterations


In [88]:
data_imp = pd.DataFrame(Zimp, columns = dati.columns, index = dati.index)

In [89]:
data_imp.apply(lambda x: np.mean(np.isnan(x)))

year        0.0
age         0.0
grade       0.0
stheight    0.0
stweight    0.0
bmi         0.0
bmipct      0.0
qnobese     0.0
qnowt       0.0
q66         0.0
q65         0.0
q8          0.0
q9          0.0
q10         0.0
q11         0.0
q12         0.0
q13         0.0
q14         0.0
q15         0.0
q16         0.0
q17         0.0
q18         0.0
q19         0.0
q20         0.0
q21         0.0
q22         0.0
q23         0.0
q24         0.0
q25         0.0
q26         0.0
           ... 
q58         0.0
q59         0.0
q60         0.0
q61         0.0
q62         0.0
q63         0.0
q64         0.0
q67         0.0
q68         0.0
q69         0.0
q70         0.0
q71         0.0
q72         0.0
q73         0.0
q74         0.0
q75         0.0
q76         0.0
q77         0.0
q78         0.0
q79         0.0
q80         0.0
q81         0.0
q82         0.0
q83         0.0
q84         0.0
q85         0.0
q86         0.0
q87         0.0
q88         0.0
q89         0.0
Length: 91, dtype: float

In [91]:
data_imp.to_csv('data_imp.csv')